# 태깅작업( Tagging ) 이란?

자연어 처리 분야에서 각 단어가 어떤 유형에 속해 있는지를 알아내는 작업을 태깅작업이라고 한다.  이러한 단어 태깅 작업은 대표적으로 크게 두가지로 나뉘는데,   

- 각 단어의 유형이 사람, 장소, 단체 등 어떤 유형인지를 알아내는 __개체명 인식(Named Entity Recognition)__
- 각 단어의 품사가 명사, 동사, 형용사 인지를 알아내는 __품사태깅 (Part-of-Speech Tagging)__

이 있다.

이러한 작업은 챗봇, 기계 번역 등과 같은 자연어 처리 분야에서 메인 작업에 앞서 전처리 작업으로서 필요한 경우가 많지만, 전처리 작업이라고 쉬운 작업이 아니라 그 자체로도 까다로운 작업이 되어 많은 시간을 필요로 한다.  

### Keras 를 이용한 태깅 작업 ( Tagging Task using Keras )

개체명 인식기와 품사 태거를 만드는데, 이러한 두 작업의 공통점은 RNN의 다-대-다 (many to many) 작업이면서 또한 앞, 뒤, 시점의 입력을 모두 참고하는 양방향 Bidirectional (RNN)을 사용한다는 것이다.  

### 훈련데이터에 대한 이해 

태깅 작업은 앞서 배운 텍스트 분류 작업과 동일하게 지도학습 (supervised Learning)에 속한다. 이 챕터에서는 태깅을 해야하는 단어 데이터를 X, 레이블에 해당되는 태깅 정보 데이터는 y라고 이름을 붙일 것이다. X에 대한 훈련 데이터는 X_train, 테스트 데이터는 X_test라고 명명하고 y에 대한 훈련 데이터는 y_train, 테스트 데이터는 y_test라고 명명합니다.  

X와 y데이터의 쌍(pair)는 병렬 구조를 가진다는 특징을 가진다. X와 y의 데이터의 길이(갯수) 는 같다.

![](img/20200331_155740.png)

가령, X_train[3]의 'The'와 y_train[3]의 'O'는 하나의 쌍(pair)입니다. 또한, X_train[3]의 'European'과 y_train[3]의 'B-ORG'는 쌍의 관계를 가지며, X_train[3]의 'Commision'과 y_train[3]의 'I-ORG'는 쌍의 관계를 가집니다.

이렇게 병렬 관계를 가지는 각 데이터는 정수 인코딩 과정을 거친 후, 모든 데이터의 길이를 동일하게 맞춰주기위한 패딩(Padding) 작업을 거칩니다.

### 시퀀스 레이블링 (Sequence Labeling) 

위와 같이 입력 시퀀스 X = [x1, x2, x3, ... , xn] 에 대하여 레이블 시퀀스 y = [y1, y2, y3, .. , yn] 를 각각 부여하는 작업을 시퀀스 레이블링 작업 (Sequence Labeling Task)라고 한다. 태깅 작업은 대표적인 시퀀스 레이블링 작업이다.  

### RNN의 다-대-다 (Many-to-Many) 문제

RNN의 은닉층은 모든 시점에 대해서 은닉 상태값을 출력할 수 도 있고, 마지막 시점에 대해서만 은닉 상태값을 출력할 수 도 있습니다.  
이는 아래  keras 코드에서 
model.add(Bidirectional(LSTM(hidden_size, return_sequences=True)))
인자로 return_sequence = True를 넣을 것인지, 넣지 않을 것인지로 설정이 가능하다.  
보통 태깅 작업의 경우 다-대-다 ( Many-to-Many ) 문제로 return_sequence = True 를 설정하여 출력층에 모든 은닉 상태 값을 나타낸다.    

이제 그림을 통하여 우리 RNN 모델이 어떻게 설계가 될지 확인해보자.

![](img/20200331_162047.png)

여기서는 양방향 RNN 을 사용하기에 아래와 같은 그림으로 표현이 되었다. 

### 개체명 인식 ( Named Entity Recognition )

개체명 인식( Named Entity Recognition ) 위에서 말했듯 이름을 가진 개체(named entity)를 인식하겠다는 것을 의미, 좀더 쉽게 설명하자면 어떤 이름을 의미하는 단어를 보고는 그 단어가 어떤 유형인지를 인식하는것,   

예를 들어 도연이는 2018년에 골드만삭스에 입사했다. 라는 문장이 있을 때, 사람(person), 조직(organization), 시간(time)에 대해 개체명 인식을 수행하는 모델이라면 다음과 같은 결과를 보여줍니다.

- 도연 - 사람
- 2018년 - 시간
- 골드만삭스 - 조직 

### NLTK 를 이용한 개체명 인식

NLTK에서는 개체명 인식기 (NER chunker)를 지원하고 있음로, 별도 개체명 인식기를 필요 없이 NLTK를 사용하여 개체명 인식을 수행할 수 있다.

In [10]:
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize, pos_tag, ne_chunk
sentence = "hi my name is seungwooHan and i live in korea"
sentence = word_tokenize(sentence)
print(sentence) # 토큰화
sentence=pos_tag(sentence)
print(sentence) # 토큰화된 문장을 품사 태깅

['hi', 'my', 'name', 'is', 'seungwooHan', 'and', 'i', 'live', 'in', 'korea']
[('hi', 'NN'), ('my', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('seungwooHan', 'JJ'), ('and', 'CC'), ('i', 'JJ'), ('live', 'VBP'), ('in', 'IN'), ('korea', 'NN')]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashgh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\ashgh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ashgh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ashgh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
sentence=ne_chunk(sentence)
print(sentence) # 개체명 인식

(S
  hi/NN
  my/PRP$
  name/NN
  is/VBZ
  (ORGANIZATION seungwooHan/JJ)
  and/CC
  i/JJ
  live/VBP
  in/IN
  korea/NN)


ne_chunk 는 개체명을 태깅하기 위해서 앞서 품사 태깅(pos_tag)이 수행되어 한다.

### 양방향 LSTM 을 이용한 개체명 인식 ( Named Entity Recognition using Bi-LSTM )

개체명 인식은 내가 하려는 __챗봇__ 등에서 필요로 하는 주요 전처리 작업이면서, 그 자체로도 까다로운 작업이다. 도메인 또는 목적에 특화되도록 개체명 인식을 정확하게 하는 방법중 하나는  
기존에 공개된 개체명 인식기를 사용하는 것이 아닌, 직접 목적에 맞는 데이터를 준비하여 기계를 훈련시켜 모델을 만드는 방법이 있다.  

### BIO 표현

개체명 인식에서 코퍼스로 부터 개체명을 인식하기 위한 방법으로는 여러 방법이 있지만, 가장 보편적인 방법으로는 IOB (또는 BIO)가 이용된다. B는 Begin의 약자로 개체명이 시작하는 부분 I는 Inside의 약자로 개체명의 내부 부분을 의미, O는 Outside의 약자로 개체명이 아닌 부분을 의미한다,   

예를 들어 영화에 대한 코퍼스 중에서 영화 제목에 대한 개체명을 뽑는다고 가정하면, 

해 B  
리 I  
포 I  
터 I  
보 O  
러 O  
가 O  
자 O  

다음과 같이 영화 제목에 대해서만 개체명을 인식할때, 영화 제목이 시작되는 글자 '해'에서는 B가 적용되고, 영화 제목이 끝나는 순간까지 I가 사용된다. 그리고 영화 제목이 아닌 부분에서 대해서 O가 사용된다. 이처럼 B와 I는 개체명을 위해 사용되고, O는 개체명이 아니라는 의미를 갖게 된다.  

물론 개체명 인식이라는 것은 보통 한 종류의 개체에 대해서만 언급하는 것이 아니라, 여러 종류의 개체가 있을 수 있다, 예를 들어 영화에 대한 대화에서는 영화 제목에 대한 개체명과 극장에 대한 개체명이 있을수 있다, 그럴때는, 각 개체가 어떤 종류인지도 함께 태깅하는 것이 좋다.  


해 B-movie  
리 I-movie  
포 I-movie  
터 I-movie  
보 O  
러 O  
메 B-theater  
가 I-theater  
박 I-theater  
스 I-theater  
가 O  
자 O  